In [ ]:
import os
import time
import json
import psutil
from PIL import Image

from character_pipeline import create_pipeline
from pipeline.base import CharacterAttributes

In [ ]:
pipeline = create_pipeline()

components = [
    'input_loader', 'preprocessor', 'edge_case_handler',
    'clip_analyzer', 'tag_parser', 'attribute_fusion',
    'rl_optimizer', 'cache_manager', 'database'
]

component_status = {}
for component in components:
    component_status[component] = hasattr(pipeline, component)

system_info = {
    'cpu_cores': psutil.cpu_count(),
    'memory_gb': round(psutil.virtual_memory().total / (1024**3), 1),
    'distributed_available': getattr(pipeline, 'distributed_available', False)
}

print('Component Status:', component_status)
print('System Info:', system_info)

In [ ]:
test_image_path = 'batch_images/pngtree-single-child-character-design-in-vector-png-image_2194494.jpg'

if os.path.exists(test_image_path):
    start_time = time.time()
    attributes = pipeline.extract_from_image(test_image_path)
    processing_time = time.time() - start_time
    
    result = {}
    for attr in ['age', 'gender', 'ethnicity', 'hair_style', 'hair_color', 
                 'hair_length', 'eye_color', 'body_type', 'dress']:
        value = getattr(attributes, attr, None)
        if value:
            result[attr.replace('_', ' ').title()] = value
    
    confidence = getattr(attributes, 'confidence_score', 0)
    
    extraction_result = {
        'processing_time': round(processing_time, 3),
        'attributes': result,
        'confidence': round(confidence, 3)
    }
    
    print('Extraction Result:', json.dumps(extraction_result, indent=2))
else:
    print('Test image not found')

In [ ]:
test_images = [
    'batch_images/pngtree-single-child-character-design-in-vector-png-image_2194494.jpg',
    'batch_images/download.jpeg',
    'batch_images/download.png'
]

preprocessing_results = []

for img_path in test_images:
    if os.path.exists(img_path):
        try:
            img = Image.open(img_path)
            preprocess_result = pipeline.preprocessor.preprocess_image(img)
            
            result = {
                'image': os.path.basename(img_path),
                'should_skip': preprocess_result['should_skip'],
                'skip_reason': preprocess_result['preprocessing_info'].get('skip_reason', 'None') if preprocess_result['should_skip'] else 'None'
            }
            
            if not preprocess_result['should_skip']:
                edge_analysis = pipeline.edge_case_handler.analyze_image_content(preprocess_result['processed_image'])
                result['edge_cases'] = edge_analysis.get('edge_cases', [])
                result['confidence'] = round(edge_analysis.get('confidence', 0), 3)
            
            preprocessing_results.append(result)
            
        except Exception as e:
            preprocessing_results.append({
                'image': os.path.basename(img_path),
                'error': str(e)
            })

print('Preprocessing Results:', json.dumps(preprocessing_results, indent=2))

In [ ]:
batch_images = []
for img_file in os.listdir('batch_images'):
    if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        batch_images.append(os.path.join('batch_images', img_file))

start_time = time.time()
batch_results = []
processed_count = 0
skipped_count = 0

for img_path in batch_images:
    try:
        attributes = pipeline.extract_from_image(img_path)
        
        if any(getattr(attributes, attr, None) for attr in ['age', 'gender', 'hair_color']):
            result = {}
            for attr in ['age', 'gender', 'hair_color', 'eye_color']:
                value = getattr(attributes, attr, None)
                if value:
                    result[attr] = value
            
            batch_results.append({
                'image': os.path.basename(img_path),
                'attributes': result,
                'success': True
            })
            processed_count += 1
        else:
            skipped_count += 1
            
    except Exception as e:
        batch_results.append({
            'image': os.path.basename(img_path),
            'error': str(e),
            'success': False
        })
        skipped_count += 1

total_time = time.time() - start_time

batch_summary = {
    'total_images': len(batch_images),
    'processed': processed_count,
    'skipped': skipped_count,
    'total_time': round(total_time, 2),
    'rate_per_second': round(len(batch_images)/total_time, 2),
    'results': batch_results
}

print('Batch Processing Summary:', json.dumps(batch_summary, indent=2))

In [ ]:
cache_dirs = ['./cache', './data/cache']
cache_analysis = {
    'total_shards': 0,
    'total_size_mb': 0,
    'shard_details': []
}

for cache_dir in cache_dirs:
    if os.path.exists(cache_dir):
        cache_files = [f for f in os.listdir(cache_dir) if f.endswith('.db')]
        cache_analysis['total_shards'] += len(cache_files)
        
        for cache_file in cache_files:
            file_path = os.path.join(cache_dir, cache_file)
            file_size = os.path.getsize(file_path)
            cache_analysis['total_size_mb'] += file_size / (1024 * 1024)
            cache_analysis['shard_details'].append({
                'file': cache_file,
                'size_kb': round(file_size / 1024, 1)
            })

cache_analysis['total_size_mb'] = round(cache_analysis['total_size_mb'], 2)

if 'total_time' in locals() and len(batch_images) > 0:
    samples_per_second = len(batch_images) / total_time
    
    scaling_projections = {}
    for scale in [100_000, 1_000_000, 5_000_000]:
        single_hours = scale / (samples_per_second * 3600)
        distributed_hours = single_hours / 8
        
        scaling_projections[f'{scale:,}_samples'] = {
            'single_machine_days': round(single_hours / 24, 1),
            'distributed_8_workers_days': round(distributed_hours / 24, 1)
        }
    
    cache_analysis['scaling_projections'] = scaling_projections
    cache_analysis['storage_5m_samples_gb'] = round(5_000_000 * 0.5 / 1024, 0)

print('Storage & Scaling Analysis:', json.dumps(cache_analysis, indent=2))

In [ ]:
ray_status = {'available': False}

try:
    import ray
    ray_status['available'] = True
    ray_status['version'] = ray.__version__
    
    if not ray.is_initialized():
        ray.init(ignore_reinit_error=True, num_cpus=min(psutil.cpu_count(), 4))
        ray_status['initialized'] = True
    else:
        ray_status['initialized'] = 'already_running'
    
    resources = ray.cluster_resources()
    available_cpus = resources.get('CPU', psutil.cpu_count())
    estimated_workers = min(int(available_cpus), 8)
    
    ray_status['cluster_resources'] = {
        'cpus': available_cpus,
        'recommended_workers': estimated_workers,
        'estimated_speedup': f'{estimated_workers}x'
    }
    
except ImportError:
    ray_status['error'] = 'Ray not installed'
except Exception as e:
    ray_status['error'] = str(e)

if 'batch_results' in locals() and batch_results:
    test_img = batch_images[0] if batch_images else test_image_path
    
    start = time.time()
    result1 = pipeline.extract_from_image(test_img)
    first_run = time.time() - start
    
    start = time.time()
    result2 = pipeline.extract_from_image(test_img)
    second_run = time.time() - start
    
    cache_performance = {
        'first_run_seconds': round(first_run, 3),
        'cached_run_seconds': round(second_run, 3),
        'speedup_factor': round(first_run / second_run, 1) if second_run > 0 else 1
    }
    
    ray_status['cache_performance'] = cache_performance

print('Distributed Processing & Cache Analysis:', json.dumps(ray_status, indent=2))

In [ ]:
production_features = {
    'modular_architecture': '11 specialized components',
    'multi_modal_processing': 'CLIP + Tag parsing + RL optimization',
    'edge_case_handling': 'Quality assessment, multi-character detection',
    'scalable_caching': '16-shard SQLite + Redis hot cache',
    'distributed_processing': 'Ray framework integration',
    'failure_recovery': 'Circuit breaker patterns',
    'streaming_processing': 'Memory-efficient large dataset handling',
    'schema_validation': 'Consistent JSON output format',
    'performance_monitoring': 'Processing metrics and benchmarking',
    'deployment_ready': 'Production server configuration'
}

demo_summary = {}

if 'batch_results' in locals():
    success_count = len([r for r in batch_results if r.get('success', False)])
    demo_summary['success_rate_percent'] = round((success_count / len(batch_results)) * 100, 1)
    demo_summary['images_processed'] = len(batch_results)
    
    if 'total_time' in locals():
        demo_summary['processing_rate_per_sec'] = round(len(batch_images)/total_time, 1)

live_demos = {
    'gradio_app': 'http://localhost:7860',
    'hugging_face': 'https://huggingface.co/spaces/cheenchan/dashverse-srinivas',
    'github': 'https://github.com/ch33nchan/dashverse.git'
}

final_summary = {
    'production_features': production_features,
    'demo_results': demo_summary,
    'live_demos': live_demos
}

print('Production Summary:', json.dumps(final_summary, indent=2))

In [ ]:
print('\n=== HuggingFace Datasets Integration Test ===')

try:
    from datasets import Dataset as HFDataset
    
    items = pipeline.input_loader.get_sample_items(3)
    
    if items:
        hf_dataset = pipeline.input_loader.create_huggingface_dataset(items)
        
        if hf_dataset:
            def test_processing_fn(batch):
                results = []
                for item_id in batch['item_id']:
                    results.append({
                        'item_id': item_id,
                        'processed': True,
                        'test_confidence': 0.95
                    })
                return {'processed_results': results}
            
            processed_dataset = pipeline.input_loader.process_with_hf_map(
                test_processing_fn,
                items=items,
                batch_size=2,
                num_proc=2
            )
            
            hf_results = {
                'original_items': len(items),
                'hf_dataset_size': len(hf_dataset),
                'processed_dataset_size': len(processed_dataset) if processed_dataset else 0,
                'hf_available': True
            }
            
            print('HuggingFace Datasets Results:', json.dumps(hf_results, indent=2))
        else:
            print('HuggingFace datasets creation failed')
    else:
        print('No sample items available for HF testing')
        
except ImportError:
    print('HuggingFace datasets not available - install with: pip install datasets')
except Exception as e:
    print(f'HuggingFace datasets error: {e}')

In [ ]:
print('\n=== PyTorch DataLoader Test ===')

try:
    import torch
    from torch.utils.data import DataLoader
    
    items = pipeline.input_loader.get_sample_items(4)
    
    if items:
        pytorch_dataset = pipeline.input_loader.create_pytorch_dataset(items)
        
        dataloader = pipeline.input_loader.create_dataloader(
            items=items,
            batch_size=2,
            shuffle=False
        )
        
        batch_count = 0
        total_items = 0
        
        for batch in dataloader:
            batch_count += 1
            total_items += len(batch['item_ids'])
            if batch_count >= 2:
                break
        
        pytorch_results = {
            'dataset_size': len(pytorch_dataset),
            'batches_processed': batch_count,
            'total_items_processed': total_items,
            'pytorch_available': True
        }
        
        print('PyTorch DataLoader Results:', json.dumps(pytorch_results, indent=2))
    else:
        print('No sample items available for PyTorch testing')
        
except ImportError:
    print('PyTorch not available - install with: pip install torch')
except Exception as e:
    print(f'PyTorch DataLoader error: {e}')

In [ ]:
print('\n=== Parquet Storage Test ===')

try:
    from pipeline.parquet_storage import ParquetStorage
    
    parquet_storage = ParquetStorage()
    
    test_data = [
        {
            'item_id': 'notebook_test_001',
            'success': True,
            'attributes': {
                'age': 'young_adult',
                'gender': 'female',
                'hair_color': 'brown'
            },
            'confidence': 0.88,
            'processing_time': 1.5
        },
        {
            'item_id': 'notebook_test_002',
            'success': True,
            'attributes': {
                'age': 'teen',
                'gender': 'male',
                'hair_color': 'black'
            },
            'confidence': 0.91,
            'processing_time': 1.3
        }
    ]
    
    storage_result = parquet_storage.store_batch_results(test_data)
    
    parquet_results = {
        'storage_success': storage_result.get('success', False),
        'records_written': storage_result.get('records_written', 0),
        'storage_type': storage_result.get('storage_type', 'unknown'),
        'parquet_available': True
    }
    
    if 'filepath' in storage_result:
        parquet_results['filepath'] = storage_result['filepath']
        parquet_results['file_size_mb'] = storage_result.get('file_size_mb', 0)
    
    print('Parquet Storage Results:', json.dumps(parquet_results, indent=2))
    
except ImportError:
    print('Parquet dependencies not available - install with: pip install pandas pyarrow')
except Exception as e:
    print(f'Parquet storage error: {e}')

In [ ]:
print('\n=== FastAPI & Celery Integration Info ===')

fastapi_info = {
    'fastapi_app_file': 'fastapi_app.py',
    'celery_tasks_file': 'celery_tasks.py',
    'endpoints': {
        'POST /extract': 'Single image processing',
        'POST /batch': 'Batch processing jobs',
        'GET /jobs/{job_id}': 'Job status monitoring',
        'GET /health': 'Health check'
    },
    'celery_tasks': {
        'extract_single_image': 'Process single image async',
        'batch_extract_images': 'Process multiple images',
        'process_dataset_directory': 'Process entire dataset'
    },
    'start_commands': {
        'fastapi_server': 'python fastapi_app.py',
        'celery_worker': 'celery -A celery_tasks worker --loglevel=info',
        'celery_monitor': 'celery -A celery_tasks flower'
    }
}

print('FastAPI & Celery Info:', json.dumps(fastapi_info, indent=2))

In [ ]:
print('\n=== Large-Scale Processing Summary ===')

implementation_summary = {
    'implemented_features': {
        'huggingface_datasets': {
            'status': 'implemented',
            'file': 'pipeline/input_loader.py',
            'methods': ['create_huggingface_dataset', 'process_with_hf_map'],
            'benefits': ['Efficient batch processing', 'Multi-process support', 'Memory streaming']
        },
        'pytorch_datasets': {
            'status': 'implemented',
            'file': 'pipeline/input_loader.py',
            'classes': ['CharacterDataset'],
            'methods': ['create_pytorch_dataset', 'create_dataloader'],
            'benefits': ['Optimized data loading', 'Custom collate functions', 'Multi-worker support']
        },
        'fastapi_endpoints': {
            'status': 'implemented',
            'file': 'fastapi_app.py',
            'endpoints': 4,
            'benefits': ['Async processing', 'REST API', 'Job management', 'Health monitoring']
        },
        'celery_tasks': {
            'status': 'implemented',
            'file': 'celery_tasks.py',
            'tasks': 3,
            'benefits': ['Background processing', 'Progress tracking', 'Task cancellation']
        },
        'parquet_storage': {
            'status': 'implemented',
            'file': 'pipeline/parquet_storage.py',
            'features': ['Columnar storage', 'Compression', 'Partitioning', 'Analytics-ready'],
            'benefits': ['Efficient storage', 'Fast queries', 'Schema validation']
        }
    },
    'production_ready': True,
    'scalability_target': '5M+ samples',
    'deployment_options': ['Single machine', 'Multi-machine cluster', 'Cloud Kubernetes']
}

print('Implementation Summary:', json.dumps(implementation_summary, indent=2))

print('\nAll large-scale processing features successfully implemented!')
print('HuggingFace datasets.map() for efficient batch inference')
print('PyTorch Dataset and DataLoader for optimized data loading')
print('FastAPI endpoints with async processing capabilities')
print('Celery task queue for background job processing')
print('Parquet storage for large-scale data export and analytics')
print('\nPipeline ready for production deployment at 5M+ sample scale!')